# Use examples of [premise](https://github.com/romainsacchi/premise)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `premise` to create life cycle invneotries databases for prospective environmental impact assessment.

This library extract useful information from IAM model output files (such as those of REMIND or IMAGE) and aligns inventories in the ecoinvent database accordingly.

Additional documentation on the moethodology is available [here](https://premise.readthedocs.io/en/latest/introduction.html).

## Requirements

* Pyhton 3.7 or above
* a user license for ecoinvent v.3

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have a open `brightway2` project with abiosphere as well as an
ecoinvent v.3 database registered in that project.

In [1]:
from premise import *
import brightway2 as bw

C:\Users\sacchi_r\AppData\Local\Continuum\miniconda3\envs\coarse\lib\site-packages\constructive_geometries\cg.py:77: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [2]:
bw.projects.set_current("import_36_for_carculator")

In [6]:
bw.databases

Databases dictionary with 7 object(s):
	biosphere3
	ecoinvent 3.5 cutoff
	ecoinvent 3.6 cutoff
	ecoinvent 3.7 cutoff
	ecoinvent_remind_Budg1100_ConvSyn_2030
	ecoinvent_remind_Budg1100_ConvSyn_2040
	ecoinvent_remind_Budg1100_ConvSyn_2045

### List of available scenarios

Some scenarios come installed with the library.
They are stored in `data/iam_ouput_files` from the root directory.
They are all within the SSP2 narrative (middle of the road).

With REMIND:
* "Base"
* "NPi"
* "NDC"
* "PkBudg1300"
* "PkBudg1100"
* "PkBudg900"

With IMAGE:
* "Base"


### Database creation from default scenarios

To create a database using REMIND's SSP2 Base scenario, from ecoinvent 3.6 for the year 2028, one would execute the following cell.
This leads to the extraction of the database, some cleanup as well as importing a few additional inventories.

In [4]:
ndb = NewDatabase(
            model="remind", # <-- name of the IAM model. Can be ´remind´ or ´image´.
            scenario="SSP2-Base", # <-- name of the scenario
            year=2028, # <-- year. Must be between 2005 and 2100.
            source_db="ecoinvent 3.6 cutoff", # <-- name of the database. Must be a string.
            source_version=3.6, # <-- version of ecoinvent. Can be 3.5, 3.6 or 3.7
                 )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


100%|█████████████████████████████████| 18121/18121 [00:00<00:00, 55058.84it/s]


Adding exchange data to activities


100%|███████████████████████████████| 615644/615644 [00:49<00:00, 12354.07it/s]


Filling out exchange data


100%|██████████████████████████████████| 18121/18121 [00:05<00:00, 3433.36it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Remove empty exchanges.

/////////////////// IMPORTING ADDITIONAL INVENTORIES ////////////////////
Extracted 1 worksheets in 0.19 seconds
Extracted 1 worksheets in 0.14 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Extracted 1 worksheets in 0.02 seconds
Extracted 1 worksheets in 0.02 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Extracted 1 worksheets in 0.01 seconds
Extracted 1 worksheets in 0.01 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Add fossil carbon dioxide storage for CCS technologies.
Extracted 1 worksheets in 0.01 seconds
Extracted 1 worksheets in 0.01 seconds
Extracted 1 worksheets in 0.02 seconds
Extracted 1 worksheets in 0.02 seconds
Extracted 1 worksheets in 0.01 seconds
Extracted 1 worksheets in 0.0

or here with ecoinvent 3.7

In [4]:
ndb = NewDatabase(
        model="remind",
        scenario="SSP2-Base",
        year=2028,
        source_db="ecoinvent 3.7 cutoff", # <-- this is NEW.
        source_version=3.7, # <-- this is NEW
                 )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


100%|████████████████████████████████| 19128/19128 [00:00<00:00, 110822.23it/s]


Adding exchange data to activities


100%|███████████████████████████████| 621717/621717 [00:43<00:00, 14329.98it/s]


Filling out exchange data


100%|██████████████████████████████████| 19128/19128 [00:03<00:00, 5199.63it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Remove empty exchanges.

/////////////////// IMPORTING ADDITIONAL INVENTORIES ////////////////////
Extracted 1 worksheets in 0.16 seconds
Extracted 1 worksheets in 0.14 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Add fossil carbon dioxide storage for CCS technologies.
Extracted 1 worksheets in 0.03 seconds
Extracted 1 worksheets in 0.01 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Add fossil carbon dioxide storage for CCS technologies.
Extracted 1 worksheets in 0.01 seconds
Extracted 1 worksheets in 0.01 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Extracted 1 worksheets in 0.02 seconds
Extracted 1 worksheets in 0.02 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Extracted 1 works

### Database creation from non-default scenarios

If you have some specific IAM scenarios you would like to build a database from, you can specify the directory to those.

**Important remark**: the name of your scenario file must start with "remind_" or "image_". For example: "remind_my_special_scenario.mif".

In [5]:
ndb = NewDatabase(
            model="remind", # <-- name of the IAM model. can be ´remind´ or ´image´
            scenario="my_special_scenario", # <-- name of the scenario
            year=2028, # <-- year
            source_db="ecoinvent 3.6 cutoff", # <-- name of the database
            source_version=3.6, # <-- version of ecoinvent
            filepath_to_iam_files=r"C:\Users\sacchi_r\Downloads\REMIND" # <-- directory of specific IAM scenario files
                 )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


100%|████████████████████████████████| 18121/18121 [00:00<00:00, 105026.84it/s]


Adding exchange data to activities


100%|███████████████████████████████| 615644/615644 [00:56<00:00, 10803.39it/s]


Filling out exchange data


100%|██████████████████████████████████| 18121/18121 [00:03<00:00, 5196.16it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Remove empty exchanges.

/////////////////// IMPORTING ADDITIONAL INVENTORIES ////////////////////
Extracted 1 worksheets in 0.15 seconds
Extracted 1 worksheets in 0.13 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Add fossil carbon dioxide storage for CCS technologies.
Extracted 1 worksheets in 0.02 seconds
Extracted 1 worksheets in 0.01 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Add fossil carbon dioxide storage for CCS technologies.
Extracted 1 worksheets in 0.01 seconds
Extracted 1 worksheets in 0.01 seconds
Extracted 1 worksheets in 0.02 seconds
Extracted 1 worksheets in 0.02 seconds
Extracted 1 worksheets in 0.01 seconds
Extracted 1 worksheets in 0.01 seconds
Extracted 1 worksheets in 0.02 seconds
Extracted 1 worksheets in 0.02 seconds
Extract

#### Additional options for database creation

By default, inventories for passenger cars and trucks are not added upon the creation of the database.
But you can add them by specifying `add_passenger_cars = True` and `add_trucks = True`. This will create invenotries for passenger cars and trucks for each region of the IAM model.

In [3]:
ndb = NewDatabase(
            model="remind", # <-- name of the IAM model
            scenario="SSP2-Base", # <-- name of the scenario
            year=2028, # <-- year
            source_db="ecoinvent 3.6 cutoff", # <-- name of the database
            source_version=3.6, # <-- version of ecoinvent
            add_passenger_cars = True, # <-- False by default
            add_trucks = True # <-- False by default
                 )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


100%|████████████████████████████████| 18121/18121 [00:00<00:00, 124399.50it/s]


Adding exchange data to activities


100%|███████████████████████████████| 615644/615644 [00:35<00:00, 17575.09it/s]


Filling out exchange data


...

0% [##############################] 100% | ETA: 00:00:00 | Item ID: 814
Total time elapsed: 00:00:00


****************** IMPORTANT BACKGROUND PARAMETERS ******************
 * The country of use is CH
 * Energy batteries produced in CN using NMC chemistry.
 * in 2028, % of renewable: 12.0%, GHG intensity per kWh: 633 g. CO2-eq.
 * electrolysis is completed by smr - natural gas.
 	 * in 2028 _________________________________________ 0.0%
 * cng is completed by biogas - sewage sludge.
 	 * in 2028 _________________________________________ 0.0%
 * diesel is completed by biodiesel - cooking oil.
 	 * in 2028 _________________________________________ 0.0%
 * *********************************************************************


0% [##############################] 100% | ETA: 00:00:00 | Item ID: 814
Total time elapsed: 00:00:00


The following datasets to import already exist in the source database. They will not be imported
+------------------------------------------+--------------------------------+----------------------------+
|                   Name                   |       Reference product        |          Location          |
+------------------------------------------+--------------------------------+----------------------------+
| Fuel tank, compressed hydrogen gas, 700b | Fuel tank, compressed hydrogen |            GLO             |
|              Cooling system              |         Cooling system         |            GLO             |
|                Manifolds                 |           Manifolds            |            GLO             |
|          Weaving, carbon cloth           |     Weaving, carbon cloth      |            GLO             |
|              Catalyst layer              |         Catalyst layer         |            GLO             |
|        drilling, deep borehole/m         |   

+------------------------------------------+--------------------------------+----------------------------+


If you want to limit the inventories of passenger cars and trucks to specific regions, you need to pass a dictionary instead, like so:

In [3]:
ndb = NewDatabase(
            model="remind", # <-- name of the IAM model
            scenario="SSP2-Base", # <-- name of the scenario
            year=2028, # <-- year
            source_db="ecoinvent 3.6 cutoff", # <-- name of the database
            source_version=3.6, # <-- version of ecoinvent
            add_passenger_cars = {"region":["EUR", "USA"]}, # <-- only for EU and US
            add_trucks = {"region": ["CHA"]} # <-- only for China
                 )

No fleet composition file is provided, hence fleet average vehicles inventories will not be produced.
No fleet composition file is provided, hence fleet average truck inventories will not be produced.

////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


100%|█████████████████████████████████| 18121/18121 [00:00<00:00, 59768.13it/s]


Adding exchange data to activities


100%|███████████████████████████████| 615644/615644 [00:41<00:00, 14691.03it/s]


Filling out exchange data


...

0% [##############################] 100% | ETA: 00:00:00 | Item ID: 866
Total time elapsed: 00:00:00
C:\Users\sacchi_r\AppData\Local\Continuum\miniconda3\envs\coarse\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
C:\Users\sacchi_r\AppData\Local\Continuum\miniconda3\envs\coarse\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


****************** IMPORTANT BACKGROUND PARAMETERS ******************
 * The country of use is CH
 * Energy batteries produced in CN using NMC chemistry.
 * in 2028, % of renewable: 12.0%, GHG intensity per kWh: 633 g. CO2-eq.
 * electrolysis is completed by smr - natural gas.
 	 * in 2028 _________________________________________ 0.0%
 * cng is completed by biogas - sewage sludge.
 	 * in 2028 _________________________________________ 0.0%
 * diesel is completed by biodiesel - cooking oil.
 	 * in 2028 _________________________________________ 0.0%
 * *********************************************************************


0% [##############################] 100% | ETA: 00:00:00 | Item ID: 866
Total time elapsed: 00:00:00


The following datasets to import already exist in the source database. They will not be imported
+------------------------------------------+--------------------------------+----------------------------+
|                   Name                   |       Reference product        |          Location          |
+------------------------------------------+--------------------------------+----------------------------+
| Hydrogen, gaseous, 25 bar, from dual flu |   Hydrogen, gaseous, 25 bar    |             CH             |
| Hydrogen, gaseous, 700 bar, from gasific |   Hydrogen, gaseous, 700 bar   |             CH             |
|              Essential BoP               |         Essential BoP          |            GLO             |
| hardwood forestry, oak, sustainable fore | wood chips, wet, measured as d |             DE             |
| production of 2 wt-% potassium iodide so | production of 2 wt-% potassium |            RER             |
| Fuel tank, compressed hydrogen gas, 700b |   

+------------------------------------------+--------------------------------+----------------------------+


### Fleet files

A last word about passenger cars: it is possible to pass a fleet composition file to generate fleet average inventories and limit those to specific regions (here, the European region).

**Important remark**: fleet avergae invenotries generation from fleet compoistion files works only with REMIND at the moment.

In [7]:
ndb = NewDatabase(
            model="remind",
            scenario="SSP2-Base", 
            year=2028,
            source_db="ecoinvent 3.6 cutoff",
            source_version=3.6, 
            add_passenger_cars={
                "fleet file": r"C:\Users\sacchi_r\Downloads\REMIND\Budg1100_ConvSyn_vintcomp.csv", # <-- this is NEW
                "region": ["EUR"]
        },
     )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


100%|████████████████████████████████| 18121/18121 [00:00<00:00, 127059.33it/s]


Adding exchange data to activities


100%|███████████████████████████████| 615644/615644 [00:46<00:00, 13132.40it/s]


Filling out exchange data


...

****************** IMPORTANT BACKGROUND PARAMETERS ******************
 * The country of use is EUR
 * Power and energy batteries produced in CN using NMC chemistry.
 * in 1996, % of renewable: 9.0%, GHG intensity per kWh: 587 g. CO2-eq.
 	 * in 1997, % of renewable: 10.0%, GHG intensity per kWh: 581 g. CO2-eq.
 	 * in 1998, % of renewable: 10.0%, GHG intensity per kWh: 576 g. CO2-eq.
 	 * in 1999, % of renewable: 10.0%, GHG intensity per kWh: 570 g. CO2-eq.
 	 * in 2000, % of renewable: 11.0%, GHG intensity per kWh: 564 g. CO2-eq.
 	 * in 2001, % of renewable: 11.0%, GHG intensity per kWh: 558 g. CO2-eq.
 	 * in 2002, % of renewable: 12.0%, GHG intensity per kWh: 552 g. CO2-eq.
 	 * in 2003, % of renewable: 12.0%, GHG intensity per kWh: 546 g. CO2-eq.
 	 * in 2004, % of renewable: 12.0%, GHG intensity per kWh: 540 g. CO2-eq.
 	 * in 2005, % of renewable: 13.0%, GHG intensity per kWh: 534 g. CO2-eq.
 	 * in 2006, % of renewable: 13.0%, GHG intensity per kWh: 528 g. CO2-eq.
 	 * in 2007,

 	 * in 1999 _________________ 0.0% _________________ 0.0%
 	 * in 2000 _________________ 0.0% _________________ 0.0%
 	 * in 2001 _________________ 0.0% _________________ 0.0%
 	 * in 2002 _________________ 0.0% _________________ 0.0%
 	 * in 2003 _________________ 0.0% _________________ 0.0%
 	 * in 2004 _________________ 1.0% _________________ 0.0%
 	 * in 2005 _________________ 1.0% _________________ 0.0%
 	 * in 2006 _________________ 1.0% _________________ 0.0%
 	 * in 2007 _________________ 1.0% _________________ 0.0%
 	 * in 2008 _________________ 1.0% _________________ 0.0%
 	 * in 2009 _________________ 1.0% _________________ 0.0%
 	 * in 2010 _________________ 1.0% _________________ 0.0%
 	 * in 2011 _________________ 1.0% _________________ 0.0%
 	 * in 2012 _________________ 1.0% _________________ 0.0%
 	 * in 2013 _________________ 1.0% _________________ 0.0%
 	 * in 2014 _________________ 1.0% _________________ 0.0%
 	 * in 2015 _________________ 1.0% _________________ 0.

0% [##############################] 100% | ETA: 00:00:00 | Item ID: 2822
Total time elapsed: 00:00:05


The following datasets to import already exist in the source database. They will not be imported
+------------------------------------------+--------------------------------+----------------------------+
|                   Name                   |       Reference product        |          Location          |
+------------------------------------------+--------------------------------+----------------------------+
| Hydrogen, gaseous, 25 bar, from dual flu |   Hydrogen, gaseous, 25 bar    |             CH             |
| Hydrogen, gaseous, 700 bar, from gasific |   Hydrogen, gaseous, 700 bar   |             CH             |
| hardwood forestry, oak, sustainable fore | wood chips, wet, measured as d |             DE             |
| production of 2 wt-% potassium iodide so | production of 2 wt-% potassium |            RER             |
| Hydrogen, gaseous, 700 bar, from ATR of  |   Hydrogen, gaseous, 700 bar   |            RER             |
| transport, pipeline, supercritical CO2,  | tr

+------------------------------------------+--------------------------------+----------------------------+


# Use case with ecospold2

The source database does not have to be from a brightway2 project.
It can be directly extracted from the bunch of ecospold2 files one gets when downloaded from the [ecoinvent website](https://ecoinvent.org).

For this, one needs to specify the argument `source_db = "ecospold"` as well as `source_file_path`, which is the directory leading to the ecospold files.

For example, here we combine the use of a specific (non-default) IAM scenario file with the use of ecospold2 files as data source (ecoinvent 3.5 in this case).

In [3]:
ndb = NewDatabase(
        model="remind",
        scenario="my_special_scenario",
        year=2028, # <-- year
        source_type="ecospold", # <--- this is NEW
        source_file_path=r"C:\Users\sacchi_r\Dropbox\Public\ecoinvent 3.5_cutoff_ecoSpold02\datasets", # <-- this is NEW
        source_version=3.5,
        filepath_to_iam_files=r"C:\Users\sacchi_r\Downloads\REMIND", # <-- directory of specific IAM scenario files
             )


////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Extracting XML data from 16022 datasets
Extracted 16022 datasets in 59.54 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying 

# Transformation functions

These functions modify the extracted database:

* **update_electricity_to_iam_data()**: alignment of regional electricity production mixes as well as efficiencies for a number of
  electricity production technologies, including Carbon Capture and Storage technologies.
* **update_vehicles()**: fuel markets that supply transport vehicles are adjusted according to the IAM projections,
  including penetration of bio- and synthetic fuels.
* **update_cement_to_iam_data()**: adjustment of technologies for cement production (dry, semi-dry, wet, with pre-heater or not),
  fuel efficiency of kilns, fuel mix of kilns (including biomass and waste fuels) and clinker-to-cement ratio.
* **update_steel_to_iam_data()**: adjustment of process efficiency, fuel mix and share of secondary steel in steel markets.

They can be applied *separately*, *consecutively* or *altogether* (using instead **.update_all()**).

In [5]:
ndb.update_all()


/////////////////// ELECTRICITY ////////////////////
Remove old electricity datasets
Create high voltage markets.
Create medium voltage markets.
Create low voltage markets.
Link activities to new electricity markets.
Log of deleted electricity markets saved in C:\Users\sacchi_r\Documents\GitHub\premise\premise\data\logs
Log of created electricity markets saved in C:\Users\sacchi_r\Documents\GitHub\premise\premise\data\logs
Log of changes in power plants efficiencies saved in C:\Users\sacchi_r\Documents\GitHub\premise\premise\data\logs
Rescale inventories and emissions for Biomass IGCC CCS
Rescale inventories and emissions for Biomass CHP
Rescale inventories and emissions for Biomass IGCC
Rescale inventories and emissions for Coal IGCC
Rescale inventories and emissions for Coal IGCC CCS
Rescale inventories and emissions for Coal PC
Rescale inventories and emissions for Coal PC CCS
Rescale inventories and emissions for Coal CHP
Rescale inventories and emissions for Gas OC
Rescale invent


Create new clinker production datasets and delete old datasets

Create new clinker market datasets and delete old datasets
Relink cement market datasets to new cement production datasets
Relink activities to new cement datasets
Relink cement production datasets to new clinker market datasets
Relink clinker market datasets to new clinker production datasets

/////////////////// STEEL ////////////////////
The IAM scenario chosen does not contain any data related to the steel sector.
Transformations related to the steel sector will be skipped.


# Export

### As a Brightway2 database

Export the modified database to brightway2

In [ ]:
ndb.write_db_to_brightway()

### As a sparse matrix representation

Or export it as a sparse matrix representation.

This will export four files:

* "A_matrix.csv": matrix coordinates and values of shape (index of activity; index of product; value) for the technosphere
* "A_matrix_index.csv": labels for indices for A matrix of shape (name of activity, reference product, unit, location, index)
* "B_matrix.csv": matrix coordinates and values of shape (index of activity; index of biosphere flow; value) for the biosphere
* "B_matrix_index.csv": labels for indices for B matrix of shape (name of biosphere flow, main compartment, sub-compartmnet, unit, index)

As a convenience, you can specifiy a directory where to store the exported matrices.
If the directory does not exist, it will be created.
If you leave it unspecified, they will be stored in **data/matrices** in the root folder of the library.

In [7]:
ndb.write_db_to_matrices(filepath=r"C:/Users/sacchi_r/Downloads/exported_matrices")

Write new database to matrix.
Matrices saved in C:\Users\sacchi_r\Downloads\exported_matrices\remind\SSP2-Base\2028.


### As a SimaPro CSV file

In [5]:
ndb.write_db_to_simapro(filepath=r"C:/Users/sacchi_r/Downloads/exported_simapro_file")

Write Simapro import file.
Simapro CSV files saved in C:\Users\sacchi_r\Downloads\exported_simapro_file\remind\SSP2-Base\2028.
